# PoC #9: Natural Language to SQL Generator 🗄️

## Business Use Case
Enable non-technical users to query databases using plain English, democratizing data access.

## Production Applications
- Business intelligence tools
- Self-service analytics
- Data exploration platforms
- Report generation automation

---

In [ ]:
!pip install -qU "ibm-watsonx-ai>=1.1.22" sqlparse
print("✅ Ready")

In [ ]:
import getpass
import sqlparse
from ibm_watsonx_ai import Credentials
from ibm_watsonx_ai.foundation_models import Model
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams

model = Model(
    model_id="ibm/granite-20b-code-instruct",
    credentials=Credentials(url="https://us-south.ml.cloud.ibm.com", api_key=getpass.getpass("API Key: ")),
    project_id=getpass.getpass("Project ID: "),
    params={GenParams.MAX_NEW_TOKENS: 300, GenParams.TEMPERATURE: 0.1}
)
print("✅ SQL generator initialized")

In [ ]:
# Mock database schema
schema = """Database Schema:

Table: customers
- customer_id (INT, PRIMARY KEY)
- name (VARCHAR)
- email (VARCHAR)
- signup_date (DATE)
- country (VARCHAR)

Table: orders
- order_id (INT, PRIMARY KEY)
- customer_id (INT, FOREIGN KEY)
- order_date (DATE)
- total_amount (DECIMAL)
- status (VARCHAR)

Table: products
- product_id (INT, PRIMARY KEY)
- product_name (VARCHAR)
- category (VARCHAR)
- price (DECIMAL)
- stock_quantity (INT)
"""

print("📊 Database schema loaded")

In [ ]:
def generate_sql(natural_language_query: str) -> dict:
    prompt = f"""{schema}

Generate a SQL query for the following request. Return ONLY the SQL query, no explanations.

Request: {natural_language_query}

SQL:"""
    
    sql = model.generate_text(prompt=prompt).strip()
    
    # Format SQL for readability
    try:
        formatted_sql = sqlparse.format(sql, reindent=True, keyword_case='upper')
    except:
        formatted_sql = sql
    
    return {
        "question": natural_language_query,
        "sql": formatted_sql
    }

In [ ]:
# Test queries
test_queries = [
    "Show me all customers who signed up in 2024",
    "What's the total revenue from orders this year?",
    "Find the top 5 best-selling products",
    "List customers from USA with orders over $1000",
    "Show me products that are out of stock"
]

print("🔍 NATURAL LANGUAGE TO SQL DEMO\n" + "="*80)

for i, query in enumerate(test_queries, 1):
    print(f"\n\n{i}. Question: {query}")
    result = generate_sql(query)
    print(f"\n   SQL:\n{result['sql']}")
    print("\n" + "-"*80)

In [ ]:
# Interactive query builder
your_question = input("\nAsk a question about the database: ")

if your_question.strip():
    result = generate_sql(your_question)
    print(f"\n{'='*80}\n📝 GENERATED SQL QUERY\n{'='*80}\n")
    print(result['sql'])
    print(f"\n{'='*80}\n\n⚠️ Always review generated SQL before executing in production!")

---

## Production Benefits
- 🚀 10x faster data access
- 👥 Empower non-technical users
- 📊 Self-service analytics
- ⏱️ Reduce data team bottleneck

### Safety Features for Production
1. **Query validation**: Check for destructive operations
2. **Read-only mode**: Restrict to SELECT queries
3. **Result limiting**: Add LIMIT clauses automatically
4. **Access control**: User-based table permissions
5. **Audit logging**: Track all generated queries

### Integration Examples
```python
# Example: Tableau integration
# Example: PowerBI connector
# Example: Slack bot for data queries
```

---